In [2]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [3]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.0.3', '2.16.1', '1.4.2', '0.13.0')

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import backend as k

In [5]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [6]:
base = base[base.price > 10]
base = base[base.price < 350000]

In [7]:
values = {'vehicleType': 'limousine', 'gearbox': 'manuell',
          'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}
base = base.fillna(value=values)

In [8]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [9]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [
                                   0, 1, 3, 5, 8, 9, 10])], remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [10]:
def criar_rede(loss):
  k.clear_session()
  regressor = Sequential()
  regressor.add(InputLayer(shape=(316,)))
  regressor.add(Dense(units=158, activation='relu',
                kernel_initializer='random_uniform'))
  regressor.add(Dense(units=158, activation='relu',
                kernel_initializer='random_uniform'))
  regressor.add(Dense(units=1, activation='linear'))
  regressor.compile(loss=loss, optimizer='adam',
                    metrics=['mean_absolute_error'])
  return regressor

Não é necessário alterar o parâmetro metrics pois ele é usado somente para  mostrar o resultado e de fato ele não é utilizado no treinamento da rede neural.

In [11]:
regressor = KerasRegressor(model=criar_rede, epochs=100, batch_size=300)

In [12]:
parametros = {
    'model__loss': ['mean_squared_error', 'mean_absolute_error']
}

In [13]:
grid_search = GridSearchCV(estimator=regressor, param_grid=parametros, cv=3)

In [14]:
grid_search = grid_search.fit(previsores, preco_real)


Epoch 1/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - loss: 76314792.0000 - mean_absolute_error: 4736.7690
Epoch 2/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 56118020.0000 - mean_absolute_error: 3980.6792
Epoch 3/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 44778200.0000 - mean_absolute_error: 3403.3384
Epoch 4/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 38474236.0000 - mean_absolute_error: 3113.8357
Epoch 5/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 35031068.0000 - mean_absolute_error: 2992.4021
Epoch 6/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 35690548.0000 - mean_absolute_error: 2932.0962
Epoch 7/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 35918944.0000 - mean_absolute_error: 3011.7131
Epoch 8/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - loss: 35504352.0000 - mean_absolute_error: 2937.2048
Epoch 9/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - loss: 34242868.0000 - mean_absolute_error: 2883.1812
Epoch 10/100
799/

In [15]:
melhores_parametros = grid_search.best_params_

In [16]:
melhores_parametros

{'model__loss': 'mean_absolute_error'}